# Homework: Deep Learning (week 8)

### Dataset

In this homework, we'll build a model for predicting if we have an image of a bee or a wasp. 
For this, we will use the "Bee or Wasp?" dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/jerzydziewierz/bee-vs-wasp) and slightly rebuilt.

You can download the dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
unzip data.zip
```

The dataset contains around 2500 images of bees and around 2100 images of wasps. 

The dataset contains separate folders for training and test sets. 

In [3]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1/255).flow_from_directory(
    # '/kaggle/input/bee-or-wasp/data/train',
    'data/train',
    class_mode='binary',
    batch_size=20,
    target_size=(150, 150)
)

test_gen = ImageDataGenerator(rescale=1/255).flow_from_directory(
    # '/kaggle/input/bee-or-wasp/data/test',
    'data/test',
    class_mode='binary',
    batch_size=20,
    target_size=(150, 150)
)

train_gen.class_indices

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


{'bee': 0, 'wasp': 1}

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=SGD(learning_rate=0.002, momentum=0.8), 
              loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
____________

In [6]:
# Fit the model to the training data
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=test_gen
)

# Save the history to a file
with open('/kaggle/working/history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

Epoch 1/10
184/184 [==============================] - 10s 51ms/step - loss: 0.6846 - accuracy: 0.5510 - val_loss: 0.6458 - val_accuracy: 0.6656
Epoch 2/10
184/184 [==============================] - 10s 53ms/step - loss: 0.6479 - accuracy: 0.6027 - val_loss: 0.6133 - val_accuracy: 0.6057
Epoch 3/10
184/184 [==============================] - 11s 59ms/step - loss: 0.5971 - accuracy: 0.6704 - val_loss: 0.5684 - val_accuracy: 0.7092
Epoch 4/10
184/184 [==============================] - 9s 50ms/step - loss: 0.5550 - accuracy: 0.7305 - val_loss: 0.5862 - val_accuracy: 0.6732
Epoch 5/10
184/184 [==============================] - 9s 51ms/step - loss: 0.5191 - accuracy: 0.7607 - val_loss: 0.5458 - val_accuracy: 0.7113
Epoch 6/10
184/184 [==============================] - 10s 53ms/step - loss: 0.4967 - accuracy: 0.7699 - val_loss: 0.5476 - val_accuracy: 0.7157
Epoch 7/10
184/184 [==============================] - 8s 46ms/step - loss: 0.4668 - accuracy: 0.7862 - val_loss: 0.5592 - val_accuracy: 0.

In [9]:
# Training data augmentation
train_gen_aug = ImageDataGenerator(
    rescale=1/255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen_aug = train_gen_aug.flow_from_directory(
    '/kaggle/input/bee-or-wasp/data/train',
    class_mode='binary',
    batch_size=20,
    target_size=(150, 150)
)

# Continue training (using augmented data from now on)
history_aug = model.fit(
    train_gen_aug,
    epochs=10,
    validation_data=test_gen
)

# Save the history_aug to a file
with open('/kaggle/working/history_aug.pkl', 'wb') as file:
    pickle.dump(history_aug.history, file)

Found 3677 images belonging to 2 classes.
Epoch 1/10
184/184 [==============================] - 30s 162ms/step - loss: 0.4963 - accuracy: 0.7794 - val_loss: 0.4780 - val_accuracy: 0.7680
Epoch 2/10
184/184 [==============================] - 33s 178ms/step - loss: 0.4905 - accuracy: 0.7737 - val_loss: 0.4580 - val_accuracy: 0.8083
Epoch 3/10
184/184 [==============================] - 30s 162ms/step - loss: 0.4822 - accuracy: 0.7710 - val_loss: 0.4955 - val_accuracy: 0.7669
Epoch 4/10
184/184 [==============================] - 30s 161ms/step - loss: 0.4810 - accuracy: 0.7822 - val_loss: 0.5398 - val_accuracy: 0.7440
Epoch 5/10
184/184 [==============================] - 32s 176ms/step - loss: 0.4749 - accuracy: 0.7876 - val_loss: 0.4748 - val_accuracy: 0.7712
Epoch 6/10
184/184 [==============================] - 32s 172ms/step - loss: 0.4704 - accuracy: 0.7849 - val_loss: 0.4691 - val_accuracy: 0.7854
Epoch 7/10
184/184 [==============================] - 33s 178ms/step - loss: 0.4644 - ac

In [1]:
import pickle
# Load the histories from a file
with open('history.pkl', 'rb') as file:
    history = pickle.load(file)
with open('history_aug.pkl', 'rb') as file:
    history_aug = pickle.load(file)

In [10]:
import pandas as pd
import numpy as np

history_df = pd.DataFrame(history)
history_aug_df = pd.DataFrame(history_aug)

In [11]:
# Question 3
# What is the median of training accuracy for all the epochs for this model?
history_df.accuracy.median()

0.7652978003025055

In [12]:
# Question 4
# What is the standard deviation of training loss for all the epochs for this model?
history_df.loss.std()

0.09792663242856825

In [13]:
# Question 5
# What is the mean of test loss for all the epochs for the model trained with augmentations?
history_aug_df.val_loss.mean()

0.47552295923233034

In [17]:
# Question 6
# What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?
history_aug_df.accuracy.loc[5:].mean()

0.7917867779731751